# Large DataFrame of Unique Words

Notebook by Melinee Her

The goal for this notebook is to create a large dataframe of all unqiue words (via their "id_word" tag) from the word-dfs of all ORACC Projects.

# Mount Google Drive folder + imports + steps

The code snippet below is to mount Google Drive files so that we can interact with our Google Drive files using the file browser or command line. Running it will give a permissions prompt.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#any necessary imports
import pandas as pd
import zipfile
from zipfile import ZipFile
import json
import requests
from tqdm import tqdm
import os
import errno
import re
import random
import numpy as np
import sys
import copy
import networkx as nx
from pathlib import Path

#Set folder for remote drive
#folder = '/content/drive/My Drive/FactGrid Cuneiform (AWCA)/people/Melinee'
folder = '/content/drive/MyDrive/Melinee/'

#importing utils for the method which downloads the current text json files
os.chdir(folder + 'network/utils/')
from utils import oracc_download

# This is a user defined module that searches through the texts to find the entities in the text that
# are people and places, to be imported as nodes into the network
os.chdir(folder + 'network/')
import rank_parser4 as rp

The steps:
1. Use the directory of the dataframes created called ORACC_DFS/PROJECT_DFS to access the words_df for all projects
2. Move id_text to be the index and merge all dataframes into a large dataframe
3. Sort by unique "id_text"

For reference, there is a smaller example at the bottom of the notebook to watch the process of cleaning and merging two dataframes at a closer level.

List of the projects - split and final

In [3]:
#projects alphabetical, starts with 'a'
p1 = ['adsd','adsd/adart1','adsd/adart2','adsd/adart3','adsd/adart5','adsd/adart6','aemw/alalakh/idrimi','aemw/amarna','akklove', 'ario',
      'armep','asbp','asbp/ninmed','asbp/rlasb','atae','atae/assur','atae/burmarina','atae/durkatlimmu',
      'atae/guzana','atae/huzirina','atae/imgurenlil','atae/kalhu','atae/mallanate','atae/marqasu',
      'atae/nineveh','atae/samal','atae/szibaniba','atae/tilbarsip','atae/tuszhan']


#projects alphabetical, starts with 'b' through 'e'
p2 = ['babcity','blms','borsippa','btmao','btto','cams','cams/akno','cams/anzu','cams/barutu','cams/etana','cams/ludlul',
      'cams/selbi','cams/tlab','ckst','cmawro','cmawro/cmawr1','cmawro/cmawr2','cmawro/cmawr3', 'cmawro/maqlu','contrib/amarna', 'ctij',
      'dcclt','dcclt/ebla','dcclt/jena','dcclt/nineveh','dcclt/signlists','dccmt','dsst', 'ecut', 'eisl','epsd2','etcsri',]


#projects alphabetical, 'g' through 'r'
p3 = ['glass','hbtin','lacost','nere','obel','obmc','obta','oimea','qcat','riao',
      'ribo','ribo/bab7scores','ribo/babylon10','ribo/babylon2','ribo/babylon3','ribo/babylon4','ribo/babylon5','ribo/babylon6',
      'ribo/babylon7','ribo/babylon8','ribo/sources','rimanum','rinap','rinap/rinap1','rinap/rinap2','rinap/rinap3',
      'rinap/rinap4','rinap/rinap5','rinap/rinap5p1','rinap/scores','rinap/sources',]


#projects alphabetical, 's' through 'x'
p4 = ['saao','saao/aebp','saao/knpp','saao/saa01','saao/saa02','saao/saa03','saao/saa04','saao/saa05','saao/saa06', 'saao/saa07','saao/saa08',
      'saao/saa09','saao/saa10','saao/saa11','saao/saa12','saao/saa13','saao/saa14','saao/saa15','saao/saa16','saao/saa17',
      'saao/saa18','saao/saa19','saao/saa20','saao/saa21','saao/saas2','suhu','tcma','tsae','xcat']

#the buried projects
p5 = ['epsd2/earlylit', 'epsd2/literary', 'epsd2/praxis', 'epsd2/praxis/liturgy','epsd2/admin/ed12', 'epsd2/admin/ed3b', 'epsd2/admin/lagash2',
      'epsd2/admin/oakk', 'epsd2/admin/oldbab', 'epsd2/admin/ur3']

p6 =  ["tcma/ali1","tcma/amarna","tcma/assur","tcma/barri","tcma/bazmusian","tcma/billa", "tcma/brak","tcma/chuera","tcma/emar",
      "tcma/fekheriye","tcma/giricano","tcma/hana","tcma/haradum","tcma/hatti","tcma/kalhu","tcma/kartn","tcma/kulishinas",
      "tcma/miscellaneous","tcma/nineveh","tcma/nippur","tcma/nuzi","tcma/qitar","tcma/rimah","tcma/suri",
      "tcma/taban","tcma/tsa1","tcma/tsh1","tcma/ugarit"]

pfinal = p1+p2+p3+p4+p5+p6

# Final DataFrame
This section of the notebook creates a final dataframe by concatenating the words-df of all projects.

In [4]:
general_path = '/content/drive/MyDrive/Melinee/ORACC_DFS/PROJECT_DFS/'
pluscsv = '-words-df.csv'

def createlargedf(lst):
  dash_proj = [word.replace("/", "-") for word in lst]
  temp = pd.DataFrame()
  for i in range(len(lst)):
    print(lst[i])                   #uncomment this line to see progress as runs
    path = general_path + dash_proj[i] + pluscsv
    if i == 0:
      temp = pd.read_csv(path, low_memory=False, index_col=False)
    else:
      df = pd.read_csv(path, low_memory=False, index_col=False)
      temp = pd.concat([temp, df], ignore_index = True).drop(columns=['number']).drop_duplicates(subset=["id_word"])
  return temp

## Creating the finaldf using the smaller sections

I found that separating the projects into smaller data frames first allows me to run and create the final df in a shorter amount of time and without going over the RAM limit.

In [ ]:
#~17 minute runtime
p1_p4= createlargedf(p1+p2+p3+p4)

In [ ]:
#~13 minute runtime
p5_p6 = createlargedf(p5+p6)

In [ ]:
finaldf = pd.concat([p1_p4, p5_p6], ignore_index=True).drop_duplicates(subset = ['id_word'])

##Creating the finaldf using the full path list
Using the createlargedf function, we can pass the entire list of the project path to get the finaldf.


In [ ]:
finaldf = createlargedf(pfinal)

adsd
adsd/adart1
adsd/adart2
adsd/adart3
adsd/adart5
adsd/adart6
aemw/alalakh/idrimi
aemw/amarna
akklove
ario
armep
asbp
asbp/ninmed
asbp/rlasb
atae
atae/assur
atae/burmarina
atae/durkatlimmu
atae/guzana
atae/huzirina
atae/imgurenlil
atae/kalhu
atae/mallanate
atae/marqasu
atae/nineveh
atae/samal
atae/szibaniba
atae/tilbarsip
atae/tuszhan
babcity
blms
borsippa
btmao
btto
cams
cams/akno
cams/anzu
cams/barutu
cams/etana
cams/ludlul
cams/selbi
cams/tlab
ckst
cmawro
cmawro/cmawr1
cmawro/cmawr2
cmawro/cmawr3
cmawro/maqlu
contrib/amarna
ctij
dcclt
dcclt/ebla
dcclt/jena
dcclt/nineveh
dcclt/signlists
dccmt
dsst
ecut
eisl
epsd2
etcsri
glass
hbtin
lacost
nere
obel
obmc
obta
oimea
qcat
riao
ribo
ribo/bab7scores
ribo/babylon10
ribo/babylon2
ribo/babylon3
ribo/babylon4
ribo/babylon5
ribo/babylon6
ribo/babylon7
ribo/babylon8
ribo/sources
rimanum
rinap
rinap/rinap1
rinap/rinap2
rinap/rinap3
rinap/rinap4
rinap/rinap5
rinap/rinap5p1
rinap/scores
rinap/sources
saao
saao/aebp
saao/knpp
saao/saa01
saao/saa

## Analyzing the finaldf DataFrame

To preview the size and unique values for each column in the finaldf

In [ ]:
for col in finaldf.columns:
  uniq = finaldf[col].unique()
  print(col + ' (' + str(len(uniq)) + '): ' + str(uniq[:3]))

lang (46): ['arc' 'akk-x-neoass' 'akk-x-mbperi']
form (372095): ['mmxx' 't' 'rmyt']
id_word (7271149): ['P522613.2.1' 'P522613.2.2' 'P522613.2.3']
label (63625): ['o 1' 'o 2' 'o 3']
id_text (117729): ['P522613' 'P522597' 'P522600']
delim (1): [nan]
gdl (7214709): [nan
 "[{'n': 'n', 'sexified': '1/2(disz)', 'form': '1/2', 'id': 'P522597.2.1.0', 'seq': [{'r': '1/2'}]}]"
 "[{'gg': 'logo', 'gdl_type': 'logo', 'group': [{'s': 'MA', 'id': 'P522597.2.2.0', 'role': 'logo', 'logolang': 'sux', 'delim': '.'}, {'s': 'NA', 'id': 'P522597.2.2.1', 'role': 'logo', 'logolang': 'sux'}]}]"]
pos (55): [nan 'n' 'N']
cf (80585): [nan 'manû' 'ṣarpu']
gw (14917): [nan 'unit' 'silver']
sense (26224): [nan 'a unit of weight' 'silver']
norm (94863): [nan 'manā' 'ṣarpu']
epos (47): [nan 'N' 'DET']
headform (455): [nan 'ib-ba-lak' 'x-š']
contrefs (1079): [nan 'P522604.16.3 P522604.17.1' 'Q007155.1.4 Q007155.1.1001']
norm0 (101809): [nan 'še' 'an']
base (68084): [nan 'ši' 'an']
morph (13592): [nan '~' '~;a']
stem (

The count of unique forms and dialects per language

In [ ]:
lang_summary = finaldf.groupby('lang').count()[['form']]
lang_summary['unique forms'] = finaldf.groupby('lang').nunique()[['form']]
lang_summary

Looking at the unique amount of words and texts in finaldf

In [ ]:
id_words = len(pd.unique(finaldf['id_word']))
id_texts = len(pd.unique(finaldf['id_text']))
print("No.of.unique words :", id_words)
print("No.of.unique texts :", id_texts)

No.of.unique words : 7271149
No.of.unique texts : 117729


Exporting the finaldf to the ORACC_DFS directory

In [ ]:
finaldf.to_csv(folder + 'ORACC_DFS/finaldf.csv')

##This workflow accomplished:
The creation of a large dataframe of total words with their metadata from words_df.

No.of.unique words : 7271149

No.of.unique texts : 117729

# EXAMPLE: Merging Two DataFrames

In this example, both the adsd and adsd-adart1 projects are to be merged.
In order to do this, we need to get and open each words_df csv file and look at the dataframes.

With the dataframes imported, we can concatenate both dataframes and drop any duplicates.

In [ ]:
#gets the path in the right form
general_path = '/content/drive/MyDrive/Melinee/ORACC_DFS/'
pluscsv = '-words-df.csv'
lst = ['adsd', 'adsd/adart1']
dash_lst = [word.replace("/", "-") for word in lst]

path1 = general_path + lst[0] + '/' + dash_lst[0] + pluscsv
path2 = general_path + lst[1] + '/' + dash_lst[1] + pluscsv
df1 = pd.read_csv(path1, low_memory=False, index_col=False)
df2 = pd.read_csv(path2, low_memory=False, index_col=False)
df1.head(5)

In [ ]:
frames = [df1, df2]
result = pd.concat(frames, ignore_index = True).drop(columns = 'number')
result

,lang,form,delim,gdl,pos,id_word,label,id_text,cf,gw,sense,norm,epos,headform,contrefs
0,akk,x,NaN,"[{'x': 'ellipsis', 'id': 'X500020.2.1.0', 'bre...",u,X500020.2.1,o 1',X500020,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,akk,x,NaN,"[{'v': 'x', 'id': 'X500020.2.2.0', 'break': 'd...",u,X500020.2.2,o 1',X500020,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,akk,LUGAL,NaN,"[{'gg': 'logo', 'gdl_type': 'logo', 'group': [...",N,X500020.2.3,o 1',X500020,šarru,king,king,šarru,N,NaN,NaN
3,akk,x,NaN,"[{'v': 'x', 'id': 'X500020.3.1.0', 'breakStart...",u,X500020.3.1,o 2',X500020,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,akk,x,NaN,"[{'v': 'x', 'id': 'X500020.3.2.0', 'statusStar...",u,X500020.3.2,o 2',X500020,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
613690,akk,TIL,NaN,"[{'gg': 'logo', 'gdl_type': 'logo', 'group': [...",N,X102910.110.4,l.e. B1,X102910,qītu,end,end,qītu,N,NaN,NaN
613691,akk,KIN,NaN,"[{'gg': 'logo', 'gdl_type': 'logo', 'group': [...",MN,X102910.110.5,l.e. B1,X102910,Ulūlu,Month VI,Month VI,Ulūlu,MN,NaN,NaN
613692,akk,MU.20.KAM₂,NaN,"[{'gg': 'logo', 'gdl_type': 'logo', 'group': [...",n,X102910.110.6,l.e. B1,X102910,NaN,NaN,NaN,NaN,NaN,NaN,NaN
613693,akk,{m}se-lu-ku,NaN,"[{'det': 'semantic', 'pos': 'pre', 'seq': [{'v...",RN,X102910.110.7,l.e. B1,X102910,Seleucus,Seleucus,Seleucus,Seleucus,RN,NaN,NaN


This cell focuses on a dataframe for all texts that have the id 'X500020'.

In the initial concatenation, there are 216 recorded texts. After dropping all duplicate rows, there are 108 texts with this id. there is unique information for each 108 inputs, however the id of the text is the same.

In [ ]:
before = result.loc[result['id_text']=='X500020']
after = result.loc[result['id_text']=='X500020'].drop_duplicates()

before.shape, after.shape

((216, 15), (108, 15))

Here is how to get a list of all unique id_texts and the number of unique texts there are in the merged "result" dataframe.

In [ ]:
unique_ids_adart = result['id_text'].unique()
print(unique_ids_adart)
len(unique_ids_adart)

1609

In this cell, we want to drop duplicates from the entire result dataframe. notice that result df has 613695 rows × 15 columns.

Once we dropped all duplicate rows and renamed the dataframe "clean", we see that it has 364669 rows × 15 columns.

In [ ]:
clean = result.drop_duplicates()
clean

,lang,form,delim,gdl,pos,id_word,label,id_text,cf,gw,sense,norm,epos,headform,contrefs
0,akk,x,NaN,"[{'x': 'ellipsis', 'id': 'X500020.2.1.0', 'bre...",u,X500020.2.1,o 1',X500020,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,akk,x,NaN,"[{'v': 'x', 'id': 'X500020.2.2.0', 'break': 'd...",u,X500020.2.2,o 1',X500020,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,akk,LUGAL,NaN,"[{'gg': 'logo', 'gdl_type': 'logo', 'group': [...",N,X500020.2.3,o 1',X500020,šarru,king,king,šarru,N,NaN,NaN
3,akk,x,NaN,"[{'v': 'x', 'id': 'X500020.3.1.0', 'breakStart...",u,X500020.3.1,o 2',X500020,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,akk,x,NaN,"[{'v': 'x', 'id': 'X500020.3.2.0', 'statusStar...",u,X500020.3.2,o 2',X500020,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364778,akk,x,NaN,"[{'x': 'ellipsis', 'id': 'X600043.8.6.0', 'bre...",u,X600043.8.6,o 6',X600043,NaN,NaN,NaN,NaN,NaN,NaN,NaN
364779,akk,x,NaN,"[{'v': 'x', 'id': 'X600043.9.1.0', 'break': 'd...",u,X600043.9.1,o 7',X600043,NaN,NaN,NaN,NaN,NaN,NaN,NaN
364780,akk,x,NaN,"[{'v': 'x', 'id': 'X600043.9.2.0', 'break': 'd...",u,X600043.9.2,o 7',X600043,NaN,NaN,NaN,NaN,NaN,NaN,NaN
364781,akk,x,NaN,"[{'v': 'x', 'id': 'X600043.9.3.0', 'break': 'd...",u,X600043.9.3,o 7',X600043,NaN,NaN,NaN,NaN,NaN,NaN,NaN


If we run this similar cell from before but with the clean df, we should get the same value of unique id_texts.

In [ ]:
unique_clean = clean['id_text'].unique()
len(unique_clean)

1609